In [1]:
import pandas as pd
import sqlite3
import pickle
#run this code on a server, laptop will not be able to handle it. loop of 11,000*8,00,000

In [ ]:
conn = sqlite3.connect("Extracted.db")
c = conn.cursor()

In [ ]:
sample = pd.read_sql("SELECT * FROM Sample", conn)

In [ ]:
metapath2 = pd.read_sql("SELECT * FROM metapath2", conn)

In [ ]:
metapath3 = pd.read_sql("SELECT * FROM metapath3", conn)

In [ ]:
metapath4 = pd.read_sql("SELECT * FROM metapath4", conn)

In [ ]:
with open('CNGG_count.pickle', 'rb') as handle:
     CNGG_count = pickle.load(handle)

In [ ]:
with open('CNG_count.pickle', 'rb') as handle:
     CNG_count = pickle.load(handle)

In [ ]:
with open('Word_count.pickle', 'rb') as handle:
     Word_count = pickle.load(handle)

In [ ]:
with open('Lemma_count.pickle', 'rb') as handle:
     Lemma_count = pickle.load(handle)

In [ ]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [ ]:
innerdict = merge_two_dicts(CNG_count, CNGG_count)

In [ ]:
from collections import defaultdict
features = defaultdict(list)
for mp2i,mp2r in metapath2.iterrows():
    node1 = mp2r['node1']
    node2 = mp2r['node2']
    
    vector = []
    
    for i,r in sample.iterrows():
        den = 0
        
        if node1 == 'Lemma':
            value1 = r['Entity_1'].split("_")[0]
            den = Lemma_count[value1]
        elif node1 == 'CNG':
            value1 = r['Entity_1'].split("_")[1]
            den = CNG_count[int(value1)]
        else:
            value1 = r['Entity_1']
            den = Word_count[value1]
            
        
        if node2 == 'Lemma':
            value2 = r['Entity_2'].split("_")[0]
        elif node2 == 'CNG':
            value2 = r['Entity_2'].split("_")[1]
        else:
            value2 = r['Entity_2']        
        
        if den == 0:
            prob12 = 0
        else:
            temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value1, value2))
            prob12 = temp.Weight[0]/den
            
            prob = prob12
        
        vector.append(prob)
    
    features[node1+'|'+node2] = vector
#Important: Word|Word column vector is our label vector ! for Mutual Information Regression

In [ ]:

for mp3i,mp3r in metapath3.iterrows():
    node1 = mp3r['node1']
    node2 = mp3r['node2']
    node3 = mp3r['node3']
    vector = []
    
    for i,r in sample.iterrows():
        den1 = 0
        den2 = 0
        
        if node1 == 'Lemma':
            value1 = r['Entity_1'].split("_")[0]
            den1 = Lemma_count[value1]
        elif node1 == 'CNG':
            value1 = r['Entity_1'].split("_")[1]
            den1 = CNG_count[int(value1)]
        else:
            value1 = r['Entity_1']
            den1 = Word_count[value1]
        
        value2 = node2
        
        if node3 == 'Lemma':
            value3 = r['Entity_2'].split("_")[0]
        elif node3 == 'CNG':
            value3 = r['Entity_2'].split("_")[1]
        else:
            value3 = r['Entity_2']
            
        den2 = innerdict[node2]
        
        if den1 == 0:
            prob12 = 0
        else:
            temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value1, value2))
            prob12 = temp.Weight[0]/den1
        
        if den2 == 0:
            prob23 = 0
        else:
            temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value2, value3))
            prob23 = temp.Weight[0]/den2
            
        prob = prob12*prob23

        vector.append(prob)
    
    features[node1 + '|' + node2 + '|'+ node3]  = vector
          

In [ ]:

for mp4i,mp4r in metapath4.iterrows():
    node1 = mp4r['node1']
    node2 = mp4r['node2']
    node3 = mp4r['node3']
    node4 = mp4r['node4']
    vector = []
    
    for i,r in sample.iterrows():
        den1 = 0
        den2 = 0
        den3 = 0
        
        if node1 == 'Lemma':
            value1 = r['Entity_1'].split("_")[0]
            den1 = Lemma_count[value1]
        elif node1 == 'CNG':
            value1 = r['Entity_1'].split("_")[1]
            den1 = CNG_count[int(value1)]
        else:
            value1 = r['Entity_1']
            den1 = Word_count[value1]
        
        value2 = node2
        
        value3 = node3
        
        if node4 == 'Lemma':
            value4 = r['Entity_2'].split("_")[0]
        elif node4 == 'CNG':
            value4 = r['Entity_2'].split("_")[1]
        else:
            value4 = r['Entity_2']
            
        den2 = innerdict[node2]
        den3 = innerdict[node3]
        
        if den1 == 0:
            prob12 = 0
        else:
            temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value1, value2))
            prob12 = temp.Weight[0]/den1
        
        if den2 == 0:
            prob23 = 0
        else:
            temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value2, value3))
            prob23 = temp.Weight[0]/den2
            
        if den3 == 0:
            prob34 = 0
        else:
            temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value3, value4))
            prob34 = temp.Weight[0]/den3
            
        
        prob = prob12*prob23*prob34

        vector.append(prob)
    
    features[node1 + '|' + node2 + '|'+ node3 + '|' +node4]  = vector
          

In [ ]:
conn1 = sqlite3.connect("Features.db")

In [ ]:
dataframe = pd.DataFrame(features, index = [i for i in range(110001)]) 

In [ ]:
dataframe.to_sql("FeaturesRaw", conn1)

In [ ]:
conn.close()
conn1.close()
